### Analyse et comprehension des fichiers

In [3]:
import matplotlib.pyplot as plt
from read_data import *
from utils import *
from fasttext_utils import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [4]:
store_full_dataset_with_features(from_scratch=False, vectorize=False)

Starting train columns : ['author', 'hindex', 'text', 'nb_paper', 'mean_coauthors_hindex', 'n_coauthors', 'core_number', 'min_coauthor_hindex', 'max_coauthor_hindex', 'pagerank', 'n_coauthors_with_hindex', 'authority', 'n_neighbors_of_neighbors', 'clustering_coef', 'modindx', 'hindex_lab', 'vector_coord_0', 'vector_coord_1']
Starting test columns : ['author', 'hindex', 'text', 'nb_paper', 'mean_coauthors_hindex', 'n_coauthors', 'core_number', 'min_coauthor_hindex', 'max_coauthor_hindex', 'pagerank', 'n_coauthors_with_hindex', 'authority', 'n_neighbors_of_neighbors', 'clustering_coef', 'vector_coord_0', 'vector_coord_1']
Ending train columns : ['author', 'hindex', 'text', 'nb_paper', 'mean_coauthors_hindex', 'n_coauthors', 'core_number', 'min_coauthor_hindex', 'max_coauthor_hindex', 'pagerank', 'n_coauthors_with_hindex', 'authority', 'n_neighbors_of_neighbors', 'clustering_coef', 'modindx', 'hindex_lab', 'vector_coord_0', 'vector_coord_1']
Ending test columns : ['author', 'hindex', 'tex

In [5]:
train, test = get_processed_data()

In [6]:
d = get_clustering_coef(test["author"])

In [7]:
get_all_number_of_coauthors_with_hindex(train["author"])

,author,n_coauthors_with_hindex
0,1964267543,5
1,2153592714,1
2,217158525,1
3,2123103677,6
4,2067710487,0
...,...,...
174236,2225897966,3
174237,2612161910,1
174238,2575614996,0
174239,2078153944,2


**distribution des données**

In [ ]:
train,n_train = get_train_data()
test,n_test = get_test_data()

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(x=train['hindex'],bins=187);

**selection of sub-dataset with same distribution than the original**

**Tout le dataset (Préalablement enregistré en csv)**

In [ ]:
# store_whole_dataset(train, "../tmp/train") # si tu n'as pas encore tout le dataset en local
# store_whole_dataset(test, "../tmp/test")

In [ ]:
data = pd.read_csv("../tmp/train_full.csv")
data.head()

**Juste 10000 données**

In [ ]:
data_path = "../tmp/data_10000.csv"
# data = preprocessing_for_fastText(10000,train)
# data.to_csv(data_path,index=None)
data = pd.read_csv(data_path)
data = small_class(data, 6)
data.head()

In [ ]:
data.describe()

In [ ]:
store_full_dataset_with_features(from_scratch=False, vectorize=False, neighborhood_level=3)

In [ ]:
train, test = get_processed_data()

Les paramètre optimaux ont été optimisés pour 10000 données dans un autre jupyter avec un temps long. Le model 1 nest plus nécessaire!

In [ ]:
model0 = fasttext.train_supervised(train_path,lr = 0.626905, dim = 12, epoch = 11, wordNgrams =3)

In [ ]:
X_train, y_train, X_test, y_test = get_numpy_data(n=174241)

In [ ]:
# train_duration = 60 # in seconds
#model1 = fasttext.train_supervised(train_path, autotuneValidationFile=test_path)

In [ ]:
model0.test(test_path)

In [ ]:
**Random forest**

In [ ]:
comp = pd.DataFrame({'paramètre':['hidden layer','learning rate','epoch','loss','ngram'],'model0':[model0.dim,model0.lr,model0.epoch,model0.loss,model0.wordNgrams],'model1':[model1.dim,model1.lr,model1.epoch,model1.loss,model0.wordNgrams]})
comp

**exploitation de model**

On est sur que le modèle applique softmax sur la représentation du mot. Nous ou choisit donc d'appliquer un autre régresseur

**Data for multiple regression model**

In [8]:
X_train, y_train, X_test, y_test = get_numpy_data(n=174241)

In [9]:
X_train_norm, X_test_norm = normalize(X_train, X_test)

**Random forest**

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

**XG_BOOST**

In [ ]:
model_xg = XGBRegressor(learning_rate=0.01, max_depth=5, objective="reg:linear", subsample=0.7, n_estimators=500)
model_xg.fit(X_train,y_train)
mod_preds = model_xg.predict(X_test)
mean_squared_error(y_test,mod_preds)

**Neural Network**

In [ ]:

from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model_nn = keras.Sequential()
model_nn.add(layers.Dense(64, kernel_initializer='uniform', input_shape=(10,)))
model_nn.add(layers.Activation('softmax'))

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

model_nn.compile(loss='mse', optimizer=optimizer)

model_nn.fit(X_train,y_train, epochs=10, batch_size=5)
mod_nn_preds = model_nn.predict(X_test)
print(len(mod_nn_preds))
mean_squared_error(y_test,mod_nn_preds)

**Lasso linear**

In [ ]:
from sklearn import linear_model
model_lasso = linear_model.Lasso(alpha=0.75, fit_intercept=False, tol=0.000000001,max_iter=10000000, positive=True)
model_lasso.fit(X_train,y_train)
mod_lasso_preds = model_lasso.predict(X_test)
mean_squared_error(y_test,mod_lasso_preds)

**Decision tree**

In [ ]:
from sklearn import tree
model_tree = tree.DecisionTreeRegressor()
model_tree.fit(X_train,y_train)
mod_tree_preds = model_tree.predict(X_test)
mean_squared_error(y_test,mod_tree_preds)

### CATBOOST

In [ ]:
from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_train, y_train)
mod_preds = model_cat.predict(X_test_norm)
mean_squared_error(y_test,mod_preds)

**Light GBM**

In [104]:
d_train = lgb.Dataset(X_train, label = y_train)
params = {}
params['learning_rate'] = 0.100
params['num_iterations'] = 500
params['boosting_type'] = 'dart'
params['objective'] = 'regression'
params['metric'] = 'rmse'
params['sub_feature'] = 0.65
params['num_leaves'] = 50
params['min_data'] = 60
params['max_depth'] = 30
clf = lgb.train(params, d_train, 100)
y_pred=clf.predict(X_test)
mean_squared_error(y_test,y_pred)

/opt/homebrew/Caskroom/miniforge/base/envs/X/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003983 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2619
[LightGBM] [Info] Number of data points in the train set: 130680, number of used features: 13
[LightGBM] [Info] Start training from score 10.056688


50.94600843329854

**test linear regression**

In [ ]:
from sklearn import linear_model

reg = linear_model.TweedieRegressor().fit(X_train,y_train)

In [ ]:
reg_pred = reg.predict(X_test)

In [ ]:
mean_squared_error(y_test,reg_pred)

**test svm**

In [ ]:
from sklearn.svm import SVC

model_svm =SVC(kernel='sigmoid', C=2)

In [ ]:
model_svm.fit(X_train,y_train)

In [ ]:
svm_pred = model_svm.predict(X_test)

In [ ]:
mean_squared_error(y_test,svm_pred)

**Test CNN**

# Soumission

In [ ]:
from submit import submit
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
# from catboost import CatBoostRegressor

model_cat = CatBoostRegressor(verbose=False)
model_forest = RandomForestRegressor()
model_xg = XGBRegressor(learning_rate=0.01, max_depth=5, objective="reg:linear", subsample=0.7, n_estimators=500)

submit(model_cat)